In [2]:
import pandas as pd

/Users/necatifurkancolak/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df_stock_exchange = pd.read_excel('../data/raw/borsa_ist.xlsx')
df_bitcoin = pd.read_excel('../data/raw/btc.xlsx')
df_gold = pd.read_excel('../data/raw/gold_prices.xlsx')
df_gov_bonds = pd.read_excel('../data/raw/gov_bonds.xlsx')
df_hpi = pd.read_excel('../data/raw/hpi.xlsx')
df_inf = pd.read_csv('../data/raw/turkish_inflation_cpi.csv',sep=";")

# Stock_Exchange:

In [4]:
df_stock_exchange = df_stock_exchange.iloc[1:].reset_index(drop=True)
df_stock_exchange.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_stock_exchange.rename(columns={'Unnamed: 1': 'Price_Stock_Ex'}, inplace=True)
df_stock_exchange['Date'] = pd.to_datetime(df_stock_exchange['Date'], errors='coerce')
df_stock_exchange['Date'] = df_stock_exchange['Date'].dt.strftime('%Y-%m')

df_stock_exchange['stock_return_mo'] = df_stock_exchange['Price_Stock_Ex'] / df_stock_exchange['Price_Stock_Ex'].shift(-1) - 1 #calculate 1 month stock return
df_stock_exchange['stock_return_pa'] = (1 + df_stock_exchange['stock_return_mo'])**(12) - 1 #calculate stock return per annum

df_stock_exchange
df_stock_exchange.to_csv('../data/processed/df_stock_exchange.csv', index=False)

In [5]:
df_stock_exchange.head()

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa
0,2024-09,282.62,-0.020619,-0.22121
1,2024-08,288.57,-0.101756,-0.724111
2,2024-07,321.26,-0.012055,-0.135442
3,2024-06,325.18,0.006874,0.085678
4,2024-05,322.96,0.041538,0.62967


# Bitcoin:

In [6]:
df_bitcoin = df_bitcoin.iloc[1:].reset_index(drop=True)
df_bitcoin.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_bitcoin.rename(columns={'Unnamed: 1': 'Price_Bitcoin'}, inplace=True)
df_bitcoin['Date'] = pd.to_datetime(df_bitcoin['Date'], errors='coerce')
df_bitcoin['Date'] = df_bitcoin['Date'].dt.strftime('%Y-%m')

df_bitcoin['btc_return_mo'] = df_bitcoin['Price_Bitcoin'] / df_bitcoin['Price_Bitcoin'].shift(-1) -1 # calculate 1 month return
df_bitcoin['btc_return_pa'] = (1 + df_bitcoin['btc_return_mo'])**(12) - 1 #calculate return per annum

df_bitcoin
df_bitcoin.to_csv('../data/processed/df_bitcoin.csv', index=False)

# Gold:

In [7]:
df_gold = df_gold.iloc[1:].reset_index(drop=True)
df_gold.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gold.rename(columns={'Unnamed: 1': 'Price_Gold'}, inplace=True)
df_gold['Date'] = pd.to_datetime(df_gold['Date'], errors='coerce')
df_gold['Date'] = df_gold['Date'].dt.strftime('%Y-%m')

df_gold['gold_return_mo'] = df_gold['Price_Gold'] / df_gold['Price_Gold'].shift(-1) -1 # calculate 1 month return
df_gold['gold_return_pa'] = (1 + df_gold['gold_return_mo'])**(12) - 1 #calculate return per annum (annualized return)

df_gold
df_gold.to_csv('../data/processed/df_gold.csv', index=False)

# Government Bonds:

In [8]:
df_gov_bonds = df_gov_bonds.iloc[1:].reset_index(drop=True)
df_gov_bonds.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gov_bonds.rename(columns={'Yield (%)': 'Gov_Bond_Yield'}, inplace=True)
df_gov_bonds['Date'] = pd.to_datetime(df_gov_bonds['Date'], errors='coerce')
df_gov_bonds['Date'] = df_gov_bonds['Date'].dt.strftime('%Y-%m')  

df_gov_bonds['bond_return_pa'] = df_gov_bonds['Gov_Bond_Yield'] / 100

df_gov_bonds.to_csv('../data/processed/df_gov_bonds.csv', index=False)

# House Price Index:

In [9]:
df_hpi = df_hpi.iloc[1:].reset_index(drop=True)

df_hpi.rename(columns={'Tarih': 'Date'}, inplace=True)
df_hpi['Date'] = pd.to_datetime(df_hpi['Date'], errors='coerce')
df_hpi['Date'] = df_hpi['Date'].dt.strftime('%Y-%m')

df_hpi = df_hpi.dropna(subset=['Date'])

df_hpi['hpi_return_mo'] = df_hpi['TP KFE TR-1'] / 100
df_hpi['hpi_return_pa'] = (1 + df_hpi['hpi_return_mo'])**(12) - 1 #calculate annualized return

#df_hpi
df_hpi.to_csv('../data/processed/df_hpi.csv', index=False)

# Inflation:

In [10]:
df_inf.head()

,Date,Inflation
0,2024-09,0.4910
1,2024-08,0.5156
2,2024-07,0.6023
3,2024-06,0.7141
4,2024-05,0.7498


# The Adjustment for Annual Returns (for Bitcoin and Gold):

In [11]:
def real_return(nom_ret, inf_rate):
    return ((1 + nom_ret) / (1 + inf_rate)) - 1

In [12]:
merged_df_btc = pd.merge(df_bitcoin, df_inf, on="Date", how="inner")
merged_df_btc["real_return_bitcoin"] = real_return(merged_df_btc["btc_return_pa"], merged_df_btc["Inflation"])
merged_df_btc = merged_df_btc.drop(columns=["Inflation"])
merged_df_btc.head()

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa,real_return_bitcoin
0,2024-09,63785.09,0.081934,1.572833,0.725575
1,2024-08,58954.67,-0.086823,-0.663752,-0.778142
2,2024-07,64559.96,0.042888,0.655212,0.033023
3,2024-06,61904.96,-0.084658,-0.65406,-0.79818
4,2024-05,67630.41,0.129642,3.318071,1.467751


In [28]:
merged_df_btc[merged_df_btc["Date"]=="2019-05"]

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa,real_return_bitcoin
64,2019-05,8503.38,0.623442,334.1525,288.248727


In [31]:
merged_df_btc[merged_df_btc["Date"]=="2021-07"]

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa,real_return_bitcoin
38,2021-07,41540.22,0.201105,8.015133,6.690781


In [32]:
merged_df_btc[merged_df_btc["Date"]=="2021-08"]

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa,real_return_bitcoin
37,2021-08,47008.53,0.131639,3.410568,2.777465


In [13]:
merged_df_gold = pd.merge(df_gold, df_inf, on="Date", how="inner")
merged_df_gold["real_return_gold"] = real_return(merged_df_gold["gold_return_pa"], merged_df_gold["Inflation"])
merged_df_gold = merged_df_gold.drop(columns=["Inflation"])
merged_df_gold.head()

,Date,Price_Gold,gold_return_mo,gold_return_pa,real_return_gold
0,2024-09,2634.58,0.052405,0.845842,0.237989
1,2024-08,2503.39,0.022794,0.31056,-0.135287
2,2024-07,2447.6,0.051939,0.836068,0.145895
3,2024-06,2326.75,-0.000249,-0.002986,-0.418346
4,2024-05,2327.33,0.017968,0.238258,-0.292343


In [14]:
merged_df_stockex = pd.merge(df_stock_exchange, df_inf, on="Date", how="inner")
merged_df_stockex["real_return_stockex"] = real_return(df_stock_exchange["stock_return_pa"], df_inf["Inflation"])
merged_df_stockex = merged_df_stockex.drop(columns=["Inflation"])
merged_df_stockex.head()

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa,real_return_stockex
0,2024-09,282.62,-0.020619,-0.22121,-0.477672
1,2024-08,288.57,-0.101756,-0.724111,-0.817967
2,2024-07,321.26,-0.012055,-0.135442,-0.460427
3,2024-06,325.18,0.006874,0.085678,-0.366619
4,2024-05,322.96,0.041538,0.62967,-0.068653


In [15]:
merged_df_bonds = pd.merge(df_gov_bonds, df_inf, on="Date", how="inner")
merged_df_bonds["real_return_bond"] = real_return(df_gov_bonds["bond_return_pa"], df_inf["Inflation"])
merged_df_bonds = merged_df_bonds.drop(columns=["Inflation"])
merged_df_bonds.head()

,Date,Gov_Bond_Yield,bond_return_pa,real_return_bond
0,2024-08,26.715,0.26715,-0.150134
1,2024-07,26.465,0.26465,-0.165578
2,2024-06,26.745,0.26745,-0.208981
3,2024-05,26.330,0.26330,-0.262995
4,2024-04,26.300,0.26300,-0.278203


In [16]:
merged_df_hpi = pd.merge(df_hpi, df_inf, on="Date", how="inner")
merged_df_hpi["real_return_hpi"] = real_return(df_hpi["hpi_return_pa"], df_inf["Inflation"])
merged_df_hpi = merged_df_hpi.drop(columns=["Inflation"])
merged_df_hpi.head()

,Date,TP KFE TR,TP KFE TR-1,TP KFE TR-3,hpi_return_mo,hpi_return_pa,real_return_hpi
0,2024-08,146.50,3.658105,34.440672,0.036581,0.539002,0.032195
1,2024-07,141.33,0.928372,38.735643,0.009284,0.117273,-0.262818
2,2024-06,140.03,1.544598,46.383023,0.015446,0.201937,-0.249867
3,2024-05,137.90,3.918613,51.338894,0.039186,0.586062,-0.074697
4,2024-04,132.70,0.920222,51.570531,0.009202,0.116191,-0.362104


# Master_Table:


In [17]:
merged_all = pd.merge(merged_df_btc, merged_df_gold, on="Date", how="inner")
merged_all = pd.merge(merged_all, merged_df_stockex, on="Date", how="inner")
merged_all = pd.merge(merged_all, merged_df_bonds, on="Date", how="inner")
merged_all = pd.merge(merged_all, merged_df_hpi, on="Date", how="inner")
merged_all.head()
merged_all.to_csv('../data/processed/master_table.csv', index=False)

# Database:

In [18]:
import sqlite3

In [19]:
conn = sqlite3.connect('database_final.db')  

In [20]:
merged_df_stockex.to_sql('STOCKEX', conn, if_exists='replace', index=False)
merged_df_btc.to_sql('BTC', conn, if_exists='replace', index=False)
merged_df_gold.to_sql('GOLD', conn, if_exists='replace', index=False)
merged_df_bonds.to_sql('BOND', conn, if_exists='replace', index=False)
merged_df_hpi.to_sql("HPI",conn, if_exists='replace', index=False)
df_inf.to_sql("INF",conn, if_exists='replace', index=False)

81

In [21]:
query = "SELECT * FROM STOCKEX"
TRIAL = pd.read_sql_query(query, conn)
TRIAL

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa,real_return_stockex
0,2024-09,282.62,-0.020619,-0.221210,-0.477672
1,2024-08,288.57,-0.101756,-0.724111,-0.817967
2,2024-07,321.26,-0.012055,-0.135442,-0.460427
3,2024-06,325.18,0.006874,0.085678,-0.366619
4,2024-05,322.96,0.041538,0.629670,-0.068653
...,...,...,...,...,...
76,2018-05,221.65,-0.134416,-0.823108,-0.842958
77,2018-04,256.07,-0.118793,-0.780752,-0.804661
78,2018-03,290.59,-0.072220,-0.593238,-0.634994
79,2018-02,313.21,-0.014753,-0.163355,-0.252595


In [22]:
conn.close()

In [27]:
df_inf[df_inf["Date"]=="2019-09"]

,Date,Inflation
60,2019-09,0.0754
